In [ ]:
import os
import random
import warnings

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torchvision import models, transforms

warnings.filterwarnings('ignore')

In [3]:
IMG_SIZE = 224

def load_images_from_folder(folder, label=None):
    images = []
    labels = []
    ids = []
    for filename in sorted(os.listdir(folder)):
        if filename.endswith('.jpg'):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                images.append(img)
                if label is not None:
                    labels.append(label)
                else:
                    ids.append(filename.split('.')[0])
    if label is not None:
        return np.array(images), np.array(labels)
    else:
        return np.array(images), ids

In [4]:
# MALE = 0, FEMALE = 1

train_female_imgs, train_female_labels = load_images_from_folder('.\\data\\gender-classification\\Train\\Female', 1)
train_male_imgs, train_male_labels = load_images_from_folder('.\\data\\gender-classification\\Train\\Male', 0)
X_train = np.concatenate([train_female_imgs, train_male_imgs], axis=0)          # Shape: (8000, 224, 224, 3)
y_train = np.concatenate([train_female_labels, train_male_labels], axis=0)      # Shape: (8000,)

val_female_imgs, val_female_labels = load_images_from_folder('.\\data\\gender-classification\\Val\\Female', 1)
val_male_imgs, val_male_labels = load_images_from_folder('.\\data\\gender-classification\\Val\\Male', 0)
X_val = np.concatenate([val_female_imgs, val_male_imgs], axis=0)                # Shape: (1000, 224, 224, 3)
y_val = np.concatenate([val_female_labels, val_male_labels], axis=0)            # Shape: (1000,)

In [ ]:
def get_dataloader(X, y, batch_size=32, shuffle=True, num_workers = 0):
    # Convert numpy arrays to PyTorch tensors
    # Change shape from (N, H, W, C) to (N, C, H, W) and scale pixel values to [0, 1]
    X_tensor = torch.from_numpy(X).permute(0, 3, 1, 2).float() / 255.0
    y_tensor = torch.from_numpy(y).long()

    # Normalize using ImageNet statistics
    mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
    X_tensor = (X_tensor - mean) / std

    # Wrap the tensors in a Dataset and create a DataLoader
    dataset = TensorDataset(X_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, 
                            num_workers=num_workers, pin_memory=True)

    return dataloader

In [ ]:
train_loader = get_dataloader(X_train, y_train)
val_loader = get_dataloader(X_val, y_val)

In [6]:
images, labels = next(iter(train_loader))
print(images.shape)  # (batch_size, 3, H, W)
print(labels.shape)  # (batch_size,)
print(images[0].shape)
print(labels[0].shape)

torch.Size([32, 3, 224, 224])
torch.Size([32])
torch.Size([3, 224, 224])
torch.Size([])


In [ ]:
# Load the pretrained ResNet-18 model
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Modify the final fully connected layer for binary classification (male/female)
model.fc = nn.Linear(model.fc.in_features, 2)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Admin/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 30.9MB/s]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# TRAIN IN ONE EPOCH FOR TESTING
for epoch in range(1):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1, Loss: 0.1289


In [ ]:
# EVALUATE
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on validation set: {100 * correct / total:.2f}%")


Accuracy on validation set: 99.15%
